# Colab환경 - 구글 드라이브 연동 및 필요한 라이브러리 다운로드

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os

# 현재 위치를 KoBART 폴더로 변경
os.chdir('/content/drive/MyDrive/KoBART-summarization-main/KoBART-summarization-main')

### 라이브러리 설치

- **pandas** : 데이터프레임 및 데이터 조작을 위한 라이브러리  

- **torch (PyTorch)** : 딥러닝 모델 구현 및 학습을 위한 라이브러리    

- **transformers** : 트랜스포머 기반 모델(BERT, GPT 등)을 사용하기 위한 라이브러리  

- **tokenizers** : 빠르고 효율적인 토크나이저 사용을 위한 라이브러리  

- **lightning (PyTorch Lightning)** : 간결한 모델 훈련 관리 및 학습 프레임워크  

- **streamlit** : 웹 애플리케이션을 빠르게 구축할 수 있는 라이브러리 (시각화용)  

- **wandb (Weights & Biases)** : 모델 학습 과정 모니터링 및 실험 관리 도구

- **loguru** : 고급 로깅 기능 제공을 위한 라이브러리  

- **rouge_score** : ROUGE 점수 계산을 위한 라이브러리 (요약 성능 평가)  



In [ ]:
# 지정된 패키지 설치 코드
#!pip install pandas
#!pip install torch==2.0.1
#!pip install transformers==4.32.1
#!pip install tokenizers==0.13.3
!pip install lightning==2.0.8
!pip install streamlit==1.26.0
!pip install wandb==0.15.9
!pip install loguru
#!pip install rouge_score

# 마무리 메시지
print("설치가 완료되었습니다.")


# KoBERT에 학습 시작

주요 옵션

- gradient_clip_val 1.0 :
기울기 클리핑 값

- max_epochs 100 :
최대 에폭 수

- checkpoint checkpoint :
학습 중 모델 가중치와 상태를 저장할 디렉터리를 지정

- accelerator gpu :
GPU를 사용

- num_gpus 1 :
사용할 GPU 수

- batch_size 16 :
배치 크기 16

- num_workers 4 :
데이터를 로드하는 병렬 작업의 수를 4로 설정합니다.

In [ ]:
"""
# 학습 시작, 기울기 최대치?, 최대 에폭, 체크포인트 생성, gpu사용(코랩), 사용 gpu수, 배치 사이즈, 벙렬 개수?
!python train.py --gradient_clip_val 1.0 \
                --max_epochs 100 \
                --checkpoint checkpoint \
                --accelerator gpu \
                --num_gpus 1 \
                --batch_size 16 \
                --num_workers 4
"""

# 학습된 ckpt파일 학습을 위한 파일로 저장, 불러오기
- 학습된 모델을 transformers 라이브러리에서 사용할 수 있는 바이너리 형식으로 저장
- 토크나이저는 기존에 작성된 파일을 다운로드 하여 사용

In [ ]:
# 학습된 모델을 바이너리로 만듦 -
#!python get_model_binary.py --model_binary '/content/drive/MyDrive/KoBART-summarization-main/KoBART-summarization-main/checkpoint/summarization_final/epoch=99-val_loss=0.000.ckpt'

In [ ]:
import torch
from transformers import PreTrainedTokenizerFast
from transformers.models.bart import BartForConditionalGeneration

# 모델 바이너리 파일 경로
model_binary_path = '/content/drive/MyDrive/KoBART-summarization-main/KoBART-summarization-main/kobart_summary'

# KoBART 모델 및 토크나이저 로드
model = BartForConditionalGeneration.from_pretrained(model_binary_path)
tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-base-v2')

### 테스트 원문을 요약해보기
인코딩 -> 자연어 생성 -> 디코더

In [ ]:
# 입력 텍스트
input_text =  """
광화문(光化門)은 서울특별시 종로구의 조선왕조 법궁인 경복궁의 남쪽에 있는 정문이다.
"임금의 큰 덕(德)이 온 나라를 비춘다"는 의미이다.
1395년에 세워졌으며, 2층 누각 구조로 되어 있다.
경복궁의 정전인 근정전으로 가기 위해 지나야 하는 문 3개 중에서 첫째로 마주하는 문이며,
둘째는 흥례문, 셋째는 근정문이다.
광화문 앞에는 지금은 도로 건설로 사라진 월대가 자리잡고 있었으며,
양쪽에는 한 쌍의 해태 조각상이 자리잡고 있다.
광화문의 석축부에는 세 개의 홍예문(虹霓門, 아치문)이 있다.
가운데 문은 임금이 다니던 문이고, 나머지 좌우의 문은 신하들이 다니던 문이었는데,
왼쪽 문은 무신이, 오른쪽 문은 문신이 출입했다.
광화문의 가운데 문 천장에는 주작이 그려져 있고, 왼쪽 문에는 거북이가,
오른쪽 문에는 천마가 그려져 있다.
"""
# 입력 텍스트를 토큰화하여 인코딩
input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=1024, truncation=True)

# 모델을 사용하여 요약 생성
summary_ids = model.generate(input_ids, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)

# 요약 결과 디코딩
summary_text = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# 요약 출력
print("\t<요약 텍스트> \n", summary_text)

# ROUGE 평가
- ROUGE-1: 45.4% - 주요 키워드는 잘 반영됨

- ROUGE-2: 17.1% - 문장 흐름과 연결성이 부족

- ROUGE-L: 45.3% - 전체 구조는 잘 유지되었으나 세부 연결성 개선 필요

In [ ]:
import pandas as pd
from rouge_score import rouge_scorer
from tqdm import tqdm

# ROUGE Scorer 준비
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# 데이터 경로
test_path = "/content/drive/MyDrive/KoBART-summarization-main/KoBART-summarization-main/data/test.tsv"
train_path = "/content/drive/MyDrive/KoBART-summarization-main/KoBART-summarization-main/data/train.tsv"

# 데이터 불러오기
def load_data(file_path):
    df = pd.read_csv(file_path, sep='\t', header=None, names=["input_text", "reference_summary"])
    return df

# 테스트 데이터 불러오기
test_data = load_data(test_path)
print("데이터 수 : ", len(test_data))

# 모델 요약 및 평가 함수
def generate_and_evaluate(input_text, reference_summary):
    # 입력 텍스트 토큰화 및 모델 요약 생성
    input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = model.generate(input_ids, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
    generated_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    # ROUGE 점수 계산
    scores = scorer.score(reference_summary, generated_summary)
    return scores, generated_summary

In [ ]:
num_samples = 1000
rouge_scores = {'rouge1': 0, 'rouge2': 0, 'rougeL': 0}

for idx, row in tqdm(test_data.iterrows(), total=min(num_samples, len(test_data))):
    if idx >= num_samples:
        break
    input_text = row['input_text']
    reference_summary = row['reference_summary']
    scores, _ = generate_and_evaluate(input_text, reference_summary)

    rouge_scores['rouge1'] += scores['rouge1'].fmeasure
    rouge_scores['rouge2'] += scores['rouge2'].fmeasure
    rouge_scores['rougeL'] += scores['rougeL'].fmeasure

# 평균 ROUGE 점수 출력
print("평균 ROUGE 점수:")
print("ROUGE-1:", rouge_scores['rouge1'] / num_samples)
print("ROUGE-2:", rouge_scores['rouge2'] / num_samples)
print("ROUGE-L:", rouge_scores['rougeL'] / num_samples)

# 최종 모델 사용
- 각 라이브러리 및 모델, 토크나이저를 로드
- summarize_text함수에 text를 넣으면 요약문을 반환

### 1. `input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=1024, truncation=True)`

- **`tokenizer.encode(input_text)`**:
  토큰 ID로 변환
  
- **`return_tensors="pt"`**:
  PyTorch 텐서로 변환, 텐서 형태로 반환되어 모델 입력으로 사용.
  
- **`max_length=1024`**:
  입력 텍스트의 최대 길이를 1024 토큰으로 제한
  
- **`truncation=True`**:
  텍스트가 max_length를 초과할 경우 초과된 부분을 자르도록 지정

---

### 2. `summary_ids = model.generate(input_ids, max_length=max_length, min_length=min_length, length_penalty=2.0, num_beams=4, early_stopping=True)`
  
- **`max_length=max_length`**:
  생성 요약 텍스트의 최대 길이
  
- **`min_length=min_length`**:
  생성 요약 텍스트의 최소 길이

- **`length_penalty=2.0`**:
  값이 높을수록 더 짧은 텍스트 생성.
  
- **`num_beams=4`**:
  num_beams는 4개의 단어 후보를 생성하여 선택 그 중에서 선택
  
- **`early_stopping=True`**:
  요약을 일찍 종료할 수 있는 기능

---

### 3. `summary_text = tokenizer.decode(summary_ids[0], skip_special_tokens=True)`

- **`tokenizer.decode(summary_ids[0])`**:
  모델이 생성한 요약 ID를 텍스트로 변환
  
- **`skip_special_tokens=True`**:
  의미있는 텍스트만 반환


In [ ]:
import torch
from transformers import PreTrainedTokenizerFast
from transformers.models.bart import BartForConditionalGeneration

# KoBART 모델 및 토크나이저 로드
tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-base-v2')
model = BartForConditionalGeneration.from_pretrained('/content/drive/MyDrive/KoBART-summarization-main/KoBART-summarization-main/kobart_summary')

def summarize_text(input_text, max_length=150, min_length=40):
    # 입력 텍스트를 토큰화하여 인코딩
    input_ids = tokenizer.encode(       # 입력 텍스트를 단어 ID로 변환
        input_text,                     # 입력 텍스트
        return_tensors="pt",            # pytorch로 변환 -> 모델에 입력하기 위해
        max_length=1024,                # 최대 길이 1024
        truncation=True                 # 최대 길이가 초과되면 텍스트 자르기
    )

    # 모델을 사용하여 요약 생성
    summary_ids = model.generate(       # 문장 생성 함수
        input_ids,                      # 인코드한 변수
        max_length=max_length,          # 요약문 최대 길이
        min_length=min_length,          # 요약문 최소 길이
        length_penalty=2.0,             # 요약문의 길이 설정 -> 값이 높을 수록 길이가 짧아짐
        num_beams=4,                    # 단어 후보의 수 -> 단어 후보 4개에서 하나를 선택 -> 클수록 정확도 향상
        early_stopping=True             # 더 이상 생성되는 단어가 없으면 일찍 종료할 수 있는 기능
    )

    # 요약 결과 디코딩
    summary_text = tokenizer.decode(    # 단어 ID를 다시 텍스트로 변환
        summary_ids[0],                 # 요약문에 해당하는 인덱스
        skip_special_tokens=True        # 의미 있는 텍스트만 변환함
    )

    return summary_text                 # 최종 요약 문장

In [ ]:
import pandas as pd
import random

# 데이터 경로
test_path = "/content/drive/MyDrive/KoBART-summarization-main/KoBART-summarization-main/data/test.tsv"

# tsv 파일 읽기
test_data = pd.read_csv(test_path, sep='\t', header=None, names=["input_text", "reference_summary"])
print("데이터 수:", len(test_data))

# 랜덤 데이터 출력 개수
num_samples = 3
print("랜덤으로 추출할 데이터 수:", num_samples, "\n\n")

# 랜덤 샘플 선택
random_rows = test_data.sample(n=num_samples)

# 랜덤 샘플에 대해 원문, 실제 요약문, 예측 요약문 출력
for index, row in random_rows.iterrows():
    input_text = row['input_text']                 # 원문 텍스트
    reference_summary = row['reference_summary']   # 실제 요약문
    result = summarize_text(input_text)            # 모델로 예측한 요약문

    # 데이터 출력 포맷 수정
    print("="*10, f"데이터 {index + 1}", "="*50)

    # 원문 출력 및 길이 출력
    print(f"\n원문 (길이: {len(input_text)} 문자)\n")
    print(input_text)
    print("-" * 200)

    # 실제 요약문 출력 및 길이 출력
    print(f"\n실제 요약문 (길이: {len(reference_summary)} 문자)\n")
    print(reference_summary)
    print("-" * 200)

    # 예측 요약문 출력 및 길이 출력
    print(f"\n예측 요약문 (길이: {len(result)} 문자)\n")
    print(result)
    print("-" * 200)

